# Automated Lead Generator

These are the packages used to make the whole thing work. Off the top of my head
I believe you have to pip install selenium, pip install beautiful soup, pip install pandas,
and I believe the rest is already in the default python package arrangement.

In [35]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np 

This allows the dataframe that's displayed to not shorten the strings so that you can see every letter of type.

In [43]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

# # First

User must run this code below in order to put together a list of names he wants to run through the code. No need
to add anything other than the first and last name. It will ask for the number of doctors first. Then firstname and then lastname over and over until the number of doctors first input is reached.

In [46]:
# creating an empty list 
firstnamelist = [] 
lastnamelist = []
tpsearchlist = []
  
# number of elemetns as input 
n = int(input("Enter number of doctors: ")) 
  
# iterating till the range 
for i in range(0, n): 
    firstname = str(input("Enter first name: "))
    lastname = str(input("Enter last name: "))
    tpsearched = "no"
    tpsearchlist.append(tpsearched)
  
    firstnamelist.append(firstname) # adding the element 
    lastnamelist.append(lastname) # adding the element
      
for i,j,k in zip(firstnamelist,lastnamelist,tpsearchlist):
    print(f"{i} {j} {k}")

Enter number of doctors: 12
Enter first name: Brian
Enter last name: Shiple
Enter first name: Qaisar
Enter last name: Shah
Enter first name: Galina
Enter last name: Simkin
Enter first name: Adam
Enter last name: Simmons
Enter first name: Gloria
Enter last name: Simms
Enter first name: Virginia
Enter last name: Simnad
Enter first name: Christopher
Enter last name: Simon
Enter first name: Alexander
Enter last name: Venizelos
Enter first name: Ming
Enter last name: Wang
Enter first name: Ayse
Enter last name: Turan
Enter first name: Omar
Enter last name: Turk
Enter first name: Scott
Enter last name: Turner
Brian Shiple no
Qaisar Shah no
Galina Simkin no
Adam Simmons no
Gloria Simms no
Virginia Simnad no
Christopher Simon no
Alexander Venizelos no
Ming Wang no
Ayse Turan no
Omar Turk no
Scott Turner no


In [88]:
doximityurllist = []
healthgradesurllist = []
middleinitiallist = []
statelist = []
certificationlistoflists = []
doctoragelist = []
doctorphonenumberslistoflists = []


secondsrest = 1
#This code uses the doctors name in google searches 
#to grab the first result when searching alongside a certain website we want
browser = webdriver.Chrome()
url = "http://www.google.com"

for firstname,lastname in zip(firstnamelist,lastnamelist):
    
    #In this portion we are grabbing the results from doximity
    #First we
    browser.get(url)
    searchform = browser.find_element_by_name("q")
    searchform.send_keys(f"Dr {firstname} {lastname} Doximity")
    searchform.submit()
    time.sleep(secondsrest)
    
    try:
        try:
            firstlinkdoximity = browser.find_element_by_xpath(f"//a[contains(.,'doximity') and contains(.,'{firstname}{lastname}')]")
        except:
            firstlinkdoximity = browser.find_element_by_xpath(f"//a[contains(.,'doximity') or contains(.,'{firstname}') or contains(.,'{lastname}')]")
        hrefdoximity = firstlinkdoximity.get_attribute("href")
        doximityurllist.append(hrefdoximity)
        #Next we want to grab the middle initial if we can get it and save that for 
        #True people search later. We will save this href as a bs4 object.
        ####THIS CODE GRABS THE Middle Initial and State
        page = requests.get(hrefdoximity)
        soup = BeautifulSoup(page.content, 'html.parser')
        middleinitialelement = soup.find("span", {"class":"user-name-middle"})
        middleinitialtext = middleinitialelement.text
        #where the list is being appended
        middleinitiallist.append(middleinitialtext)


        stateelement = soup.find("span", {"itemprop":"addressRegion"})
        statetext = stateelement.text
        #where the list is being appended
        statelist.append(statetext)
        ####THIS CODE GRABS THE CERTIFICATION LIST

        certificationsection = soup.find("section", {"class":"section certification-info"})
        certificationlist = certificationsection.find_all("strong")
        #This removes the heading
        certificationlistfixed = certificationlist[1:]
        certificationlisttext = []
        for item in certificationlistfixed:
            certificationlisttext.append(item.text)    
        #where the list is being appended to a greater list
        certificationlistoflists.append(certificationlisttext)
        
    except:
        middleinitialtext = "none"
        statetext = "none"
        certificationlisttext = "none"
        middleinitiallist.append(middleinitialtext)
        statelist.append(statetext)
        certificationlistoflists.append(certificationlisttext)
    
    time.sleep(secondsrest)
    
    #In this portion we are grabbing the results from healthgrades
    
    browser.get(url)
    searchform = browser.find_element_by_name("q")
    if middleinitialtext == "none" and statetext == "none":
        searchform.send_keys(f"Dr {firstname} {lastname} Healthgrades")
        searchform.submit()   
    elif middleinitialtext == "none":
        searchform.send_keys(f"Dr {firstname} {lastname} Healthgrades {statetext}")
        searchform.submit() 
    elif statetext == "none":
        searchform.send_keys(f"Dr {firstname} {middleinitialtext} {lastname} Healthgrades")
        searchform.submit() 
    else:
        searchform.send_keys(f"Dr {firstname} {middleinitialtext} {lastname} Healthgrades {statetext}")
        searchform.submit()
    time.sleep(secondsrest)
    try:
        firstlinkhealthgrades = browser.find_element_by_xpath(f"//a[contains(.,'healthgrades') and contains(.,'{lastname}')]")
        hrefhealthgrades = firstlinkhealthgrades.get_attribute("href")
        healthgradesurllist.append(hrefhealthgrades)
        ####THIS CODE GRABS THE age and locations
        page = requests.get(hrefhealthgrades)
        soup = BeautifulSoup(page.content, 'html.parser')
        doctorageelement = soup.find("span", {"data-qa-target":"ProviderDisplayAge"})
        doctoragetext = doctorageelement.text[-2:]
        #Where the item is being appended to a list
        doctoragelist.append(doctoragetext)
        
    except:
        hrefhealthgrades = "none"
        healthgradesurllist.append(hrefhealthgrades)
        doctoragelist.append("none")
    
    time.sleep(secondsrest)
    
browser.close()

In [89]:
DoctorDataDictionary = {'firstname':firstnamelist,
                        'lastname':lastnamelist,
                        'doximityurl':doximityurllist,
                        'healthgradesurl':healthgradesurllist,
                        'middleinitial':middleinitiallist,
                        'state':statelist,
                        'certifications':certificationlistoflists,
                        'age': doctoragelist,
                        'phone numbers': doctorphonenumberslistoflists,
                        'tpsearched?': tpsearchlist}
DoctorDataDictionary

{'firstname': ['Brian',
  'Qaisar',
  'Galina',
  'Adam',
  'Gloria',
  'Virginia',
  'Christopher',
  'Alexander',
  'Ming',
  'Ayse',
  'Omar',
  'Scott'],
 'lastname': ['Shiple',
  'Shah',
  'Simkin',
  'Simmons',
  'Simms',
  'Simnad',
  'Simon',
  'Venizelos',
  'Wang',
  'Turan',
  'Turk',
  'Turner'],
 'doximityurl': ['https://www.doximity.com/pub/brian-shiple-do',
  'https://www.doximity.com/cv/qaisar-shah-md-1',
  'https://www.doximity.com/pub/galina-simkin-md',
  'https://www.doximity.com/pub/adam-simmons-md',
  'https://www.doximity.com/pub/gloria-simms-md',
  'https://www.doximity.com/pub/virginia-simnad-md',
  'https://www.doximity.com/pub/christopher-simon-md',
  'https://www.doximity.com/pub/alexander-venizelos-md',
  'https://www.doximity.com/pub/ming-wang-md-0686af52',
  'https://www.doximity.com/pub/ayse-turan-md',
  'https://www.doximity.com/pub/omar-turk-md',
  'https://www.doximity.com/pub/scott-turner-md-314b4f76'],
 'healthgradesurl': ['https://www.healthgrades.c

In [90]:
df=pd.DataFrame.from_dict(DoctorDataDictionary,orient='index').transpose()
df

,firstname,lastname,doximityurl,healthgradesurl,middleinitial,state,certifications,age,phone numbers,tpsearched?
0,Brian,Shiple,https://www.doximity.com/pub/brian-shiple-do,https://www.healthgrades.com/physician/dr-brian-j-shiple-yjmvm,J,PA,"[LA State License, DE State License, PA State License]",60,None,yes
1,Qaisar,Shah,https://www.doximity.com/cv/qaisar-shah-md-1,https://www.healthgrades.com/physician/dr-qaisar-shah-x6xmm,A,PA,"[NY State Medical License, GA State Medical License, FL State Medical License, NJ State Medical License, PA State Medical License, American Board of Psychiatry and Neurology]",44,None,yes
2,Galina,Simkin,https://www.doximity.com/pub/galina-simkin-md,https://www.healthgrades.com/physician/dr-galina-simkin-xw8n9,B,IL,"[OH State Medical License, WI State Medical License, FL State Medical License, NH State Medical License, IL State Medical License, American Board of Psychiatry and Neurology]",60,None,yes
3,Adam,Simmons,https://www.doximity.com/pub/adam-simmons-md,https://www.healthgrades.com/physician/dr-adam-simmons-3dmrd,D,MA,"[CT State Medical License, MA State Medical License, American Board of Psychiatry and Neurology]",46,None,yes
4,Gloria,Simms,https://www.doximity.com/pub/gloria-simms-md,https://www.healthgrades.com/physician/dr-gloria-simms-gcpbn,Patricia,TX,"[TX State Medical License, KY State Medical License, ME State Medical License, AL State Medical License, WI State Medical License, IA State Medical License, MN State Medical License, American Board of Psychiatry and Neurology]",45,None,yes
5,Virginia,Simnad,https://www.doximity.com/pub/virginia-simnad-md,https://www.healthgrades.com/physician/dr-virginia-simnad-2xkyj,I,WA,"[WA State Medical License, VA State Medical License, American Board of Psychiatry and Neurology]",59,None,yes
6,Christopher,Simon,https://www.doximity.com/pub/christopher-simon-md,https://www.healthgrades.com/physician/dr-christopher-simon-xcm76,P,IL,"[IL State Medical License, American Board of Psychiatry and Neurology, American Board of Psychiatry and Neurology]",47,None,yes
7,Alexander,Venizelos,https://www.doximity.com/pub/alexander-venizelos-md,https://www.healthgrades.com/physician/dr-alexander-venizelos-xf7fm,P,OH,"[OH State Medical License, TX State Medical License, IL State Medical License, American Board of Psychiatry and Neurology]",38,None,yes
8,Ming,Wang,https://www.doximity.com/pub/ming-wang-md-0686af52,https://www.healthgrades.com/physician/dr-ming-wang-yl8xh,none,none,none,58,None,yes
9,Ayse,Turan,https://www.doximity.com/pub/ayse-turan-md,none,none,none,none,none,None,yes


In [67]:
tpsearchlist[0] = 'no'
tpsearchlist[0]

'no'

In [83]:
for firstname,lastname,statetext,tpsearchstatus in zip(firstnamelist,lastnamelist, statelist, tpsearchlist):
    if tpsearchstatus == "no":
            print(f"{firstname} {lastname} {statetext} {tpsearchstatus}")

Omar Turk MI no
Scott Turner MO no


In [91]:
secondsrest = 1
#This code uses the doctors name in google searches 
#to grab the first result when searching alongside a certain website we want
browser = webdriver.Chrome()
url = "http://www.google.com"

#In this portion we are grabbing the results from truepeoplesearch
#THIS IS WHERE WE WANT TO SEPERATE FROM THE LOOP SO WE DONT
#GET STUCK ON CAPTCHAs from rehitting the sight. The plan is to use the searchbarwithin the site
#once we're inside it and loop through the names from there
browser.get(url)
searchform = browser.find_element_by_name("q")
searchform.send_keys("True People Search")
searchform.submit()
time.sleep(secondsrest)
browser.find_element_by_xpath('//*[@id="rso"]/div[1]/div/div/div/div/div[1]/a').click()

#We are now inside the site and safe to loop through the names from here
   

for doctorname,statetext,tpsearchstatus in zip(doctorlist, statelist, tpsearchlist):
  
    if tpsearchstatus == 'yes':
        searchname = browser.find_element_by_xpath('//*[@id="Name"]')
        searchname.send_keys(f"{doctorname}")
        searchstate = browser.find_element_by_xpath('//*[@id="CityStateZip"]')
        if statetext == "none":
            searchname.submit()
        else:
            searchstate.send_keys(f"{statetext}")
            searchname.submit()
        time.sleep(secondsrest)
        #This clicks first result
        try: 
            #this should have an includes text xpath for more accurate selection. Right now it only takes the first result.
            browser.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[4]/div[1]/div[2]/a').click()
            try:
                #This clicks for more phones
                browser.find_element_by_xpath('//*[@id="divMorePhones"]/div/button').click()
            except:
                pass

            hreftruepeoplesearch = browser.current_url
            #This code grabs the phone numbers from truepeoplesearch
            page = requests.get(hreftruepeoplesearch)
            soup = BeautifulSoup(page.content, 'html.parser')
            phoneelements = soup.find_all("a", {"data-link-to-more":"phone"})
            phonelisttext = []
            for item in phoneelements:
                phonelisttext.append(item.text)    
            #where the list is being appended to an even greater list
            doctorphonenumberslistoflists.append(phonelisttext)
        except:
            browser.find_element_by_xpath('//*[@id="Name"]').clear()
            browser.find_element_by_xpath('//*[@id="CityStateZip"]').clear()
            tpsearchstatus = "yes"

        browser.find_element_by_xpath('//*[@id="Name"]').clear()
        browser.find_element_by_xpath('//*[@id="CityStateZip"]').clear()
        
        time.sleep(secondsrest)
    else:
        deletethis = "none"

browser.close()



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="Name"]"}
  (Session info: chrome=77.0.3865.90)


In [5]:
for doctorname, statetext in zip(doctorlist, statelist):
    print(f"{doctorname} and {statetext}")
  

Qaisar Shah and PA
Adam Simmons and MA
Gloria Simms and TX
Virginia Simnad and WA
Christopher Simon and IL
Alexander Venizelos and OH
Ming Wang and none
Ayse Turan and none
Omar Turk and MI
Scott Turner and MO
Brian Shiple and PA


In [55]:
DoctorDataDictionary = {'doctorname':doctorlist,
                        'middleinitial':middleinitiallist,
                        'state':statelist,
                        'certifications':certificationlistoflists,
                        'age': doctoragelist,
                        'phone numbers': doctorphonenumberslistoflists}
DoctorDataDictionary

{'doctorname': ['Qaisar Shah',
  'Adam Simmons',
  'Gloria Simms',
  'Virginia Simnad',
  'Christopher Simon',
  'Alexander Venizelos',
  'Ming Wang',
  'Ayse Turan',
  'Omar Turk',
  'Scott Turner',
  'Brian Shiple'],
 'middleinitial': ['J',
  'A',
  'B',
  'D',
  'Patricia',
  'I',
  'P',
  'P',
  'none',
  'none',
  'Amin',
  'George'],
 'state': ['PA',
  'PA',
  'IL',
  'MA',
  'TX',
  'WA',
  'IL',
  'OH',
  'none',
  'none',
  'MI',
  'MO'],
 'certifications': [['LA State License',
   'DE State License',
   'PA State License'],
  ['NY State Medical License',
   'GA State Medical License',
   'FL State Medical License',
   'NJ State Medical License',
   'PA State Medical License',
   'American Board of Psychiatry and Neurology'],
  ['OH State Medical License',
   'WI State Medical License',
   'FL State Medical License',
   'NH State Medical License',
   'IL State Medical License',
   'American Board of Psychiatry and Neurology'],
  ['CT State Medical License',
   'MA State Medica

In [43]:
page = requests.get("https://www.healthgrades.com/physician/dr-galina-simkin-xw8n9")
soup = BeautifulSoup(page.content, 'html.parser')
doctorageelement = soup.find("span", {"data-qa-target":"ProviderDisplayAge"})
doctoragetext = doctorageelement.text[-2:]
doctoragetext

'60'

In [17]:
page = requests.get()
soup = BeautifulSoup(page.content, 'html.parser')
phoneelements = soup.find_all("a", {"data-link-to-more":"phone"})
phonelisttext = []
for item in phoneelements:
    phonelisttext.append(item.text)    
phonelisttext

['(215) 860-2401',
 '(267) 722-8480',
 '(603) 443-9646',
 '(215) 385-4578',
 '(603) 767-6090',
 '(718) 591-8505']

In [18]:
page = requests.get("https://www.doximity.com/pub/ayse-turan-md")
soup = BeautifulSoup(page.content, 'html.parser')
stateelement = soup.find("span", {"itemprop":"addressRegion"})
statetext = stateelement.text
statetext

'PA'

In [71]:
hrefhealthgrades

'https://www.healthgrades.com/physician/dr-qaisar-shah-x6xmm'

In [60]:
hrefdoximity

'https://www.doximity.com/cv/qaisar-shah-md-1'

In [65]:
####THIS CODE GRABS THE Middle Initial and State
page = requests.get(hrefdoximity)
soup = BeautifulSoup(page.content, 'html.parser')
middleinitialelement = soup.find("span", {"class":"user-name-middle"})
middleinitialtext = middleinitialelement.text
middleinitialtext

stateelement = soup.find("span", {"itemprop":"addressRegion"})
statetext = stateelement.text
statetext
# certlist = certsection.find_all("strong")
# certlistfixed = certlist[1:]
# certlisttext = []
# for item in certlistfixed:
#     certlisttext.append(item.text)
# certlisttext

'PA'

In [66]:
####THIS CODE GRABS THE CERTIFICATIONS
page = requests.get(hrefdoximity)
soup = BeautifulSoup(page.content, 'html.parser')
certsection = soup.find("section", {"class":"section certification-info"})

certlist = certsection.find_all("strong")
certlistfixed = certlist[1:]
certlisttext = []
for item in certlistfixed:
    certlisttext.append(item.text)
certlisttext

['NY State Medical License',
 'GA State Medical License',
 'FL State Medical License',
 'NJ State Medical License',
 'PA State Medical License',
 'American Board of Psychiatry and Neurology']

In [18]:
soup.select("ul strong")

[<strong itemprop="name">U Minnesota</strong>,
 <strong itemprop="name">U Penn</strong>,
 <strong itemprop="name">Dartmouth-Hitchcock Medical Center</strong>,
 <strong itemprop="name">Flushing Hospital Medical Center</strong>,
 <strong itemprop="name">Dow University of Health Sciences</strong>,
 <strong>NY State Medical License</strong>,
 <strong>GA State Medical License</strong>,
 <strong>FL State Medical License</strong>,
 <strong>NJ State Medical License</strong>,
 <strong>PA State Medical License</strong>,
 <strong>American Board of Psychiatry and Neurology</strong>,
 <strong itemprop="award">Top Doctors</strong>,
 <strong itemprop="award">CMS Meaningful Use Stage 1 Certification</strong>,
 <strong itemprop="award">Super Doctor</strong>,
 <strong><a href="http://clinicaltrials.gov/ct2/show/NCT02034058">Post Market Surveillance Study of the Wingspan Stent System</a></strong>,
 <strong><a href="http://clinicaltrials.gov/ct2/show/NCT02040259">Trevo® Retriever Registry Post Market Surv

In [24]:
soupcontent = soup.find_all('section', class_='section certification-info')[0]
souptwo = BeautifulSoup(soupcontent, 'html.parser')
print(souptwo.prettify())

TypeError: 'NoneType' object is not callable

In [11]:
hrefhealthgrades

'https://www.healthgrades.com/physician/dr-qaisar-shah-x6xmm'

In [12]:

browser = webdriver.Chrome()
browser.get("http://www.google.com")
searchform = browser.find_element_by_xpath('//*[@title="Search"]')
searchform.send_keys(f"Dr {doctorname} Doximity")
searchform.submit()

In [3]:
search_form = browser.find_element_by_id('search_form_input_homepage')
search_form.send_keys('real python')
search_form.submit()
elem = driver.find_element_by_id("email")
elem.send_keys(user)
elem = driver.find_element_by_id("pass")
elem.send_keys(pwd)
elem.send_keys(Keys.RETURN)
driver.close()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 74
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Mac OS X 10.13.6 x86_64)


In [1]:
# creating an empty list 
doctorlist = [] 
  
# number of elemetns as input 
n = int(input("Enter number of doctors : ")) 
  
# iterating till the range 
for i in range(0, n): 
    name = str(input()) 
  
    doctorlist.append(name) # adding the element 
      
print(doctorlist) 

Enter number of doctors : 5
Galina Simkin
Adam Simmons
Gloria Simms
Virginia Simnad
Christopher Simon
['Galina Simkin', 'Adam Simmons', 'Gloria Simms', 'Virginia Simnad', 'Christopher Simon']
